<a href="https://colab.research.google.com/github/allispaul/audiobot/blob/main/AG_XGBoost_med.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!pip install scikit-learn-intelex

  Using cached scikit_learn_intelex-2024.3.0-py310-none-manylinux1_x86_64.whl (137 kB)
  Using cached daal4py-2024.3.0-py310-none-manylinux1_x86_64.whl (10.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 MB 27.0 MB/s eta 0:00:00


In [110]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from IPython.display import Audio

import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

from sklearnex import patch_sklearn  # Intel CPU acceleration for sklearn
patch_sklearn()
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss, accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.model_selection import GridSearchCV

cuda


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [12]:
# Directories
METADATA_DIR = Path("/content/drive/MyDrive/Audiobots/Data/fma_metadata")
DATA_DIR = Path("/content/drive/MyDrive/Audiobots/Data/")

In [51]:
# Tracks in data subsets
tracks = pd.read_csv(METADATA_DIR / "tracks.csv", index_col=0, header=[0,1])
tracks_med = tracks[tracks["set"]["subset"]=="medium"]
#tracks_med["track"]["genre_top"]

In [52]:
# FMA-Medium
fma_features = pd.read_csv(DATA_DIR / "fma_features_3sec.csv",index_col=0, header=[0,1,2,3])

# Restrict to medium dataset
fma_med = fma_features.loc[tracks_med.index]
fma_med.head()

feature        chroma_cens                                                    \
statistics             max                                                     
feature_number          01                                                     
song_pos            pos_01    pos_02    pos_03    pos_04    pos_05    pos_06   
track_id                                                                       
3                 0.417433  0.350428  0.376733  0.508503  0.443255  0.581191   
134               0.446382  0.442526  0.419414  0.433643  0.407400  0.501106   
136               0.275723  0.390478  0.269860  0.068231  0.104821  0.126996   
139               0.286037  0.731147  0.536617  0.612791  0.633770  0.646182   
181               0.401898  0.413877  0.458279  0.439906  0.449719  0.421898   

feature                                                 ...       zcr  \
statistics                                              ...       std   
feature_number                                          ...        01   
song_pos          pos_07    pos_08    pos_09    pos_10  ...    pos_01   
track_id                                                ...             
3               0.482706  0.474355  0.573654  0.513427  ...  0.078393   
134             0.421602  0.437058  0.451682  0.456216  ...  0.076696   
136             0.220359  0.307832  0.263045  0.363216  ...  0.007586   
139             0.473008  0.495474  0.437456  0.467358  ...  0.021546   
181             0.414196  0.467424  0.431753  0.399964  ...  0.035982   

feature                                                                     \
statistics                                                                   
feature_number                                                               
song_pos          pos_02    pos_03    pos_04    pos_05    pos_06    pos_07   
track_id                                                                     
3               0.057437  0.076173  0.076854  0.089157  0.055686  0.035806   
134             0.063500  0.042880  0.046147  0.050985  0.045815  0.037073   
136             0.008543  0.006473  0.008653  0.009003  0.008523  0.008338   
139             0.027704  0.028357  0.026644  0.020512  0.021097  0.016603   
181             0.033205  0.021251  0.032047  0.017333  0.019791  0.023580   

feature                                       
statistics                                    
feature_number                                
song_pos          pos_08    pos_09    pos_10  
track_id                                      
3               0.013286  0.039492  0.067383  
134             0.066570  0.045812  0.062895  
136             0.009464  0.008612  0.008036  
139             0.016951  0.011645  0.018713  
181             0.021319  0.028439  0.033835  

[5 rows x 3700 columns]

In [53]:
# Add genre classification to fma_med using tracks_med
label2idx = LabelEncoder().fit(tracks_med["track", "genre_top"])
fma_med["track", "genre_top", "", ""] = label2idx.transform(tracks_med["track", "genre_top"])

In [54]:
fma_med.head(5)

feature        chroma_cens                                                    \
statistics             max                                                     
feature_number          01                                                     
song_pos            pos_01    pos_02    pos_03    pos_04    pos_05    pos_06   
track_id                                                                       
3                 0.417433  0.350428  0.376733  0.508503  0.443255  0.581191   
134               0.446382  0.442526  0.419414  0.433643  0.407400  0.501106   
136               0.275723  0.390478  0.269860  0.068231  0.104821  0.126996   
139               0.286037  0.731147  0.536617  0.612791  0.633770  0.646182   
181               0.401898  0.413877  0.458279  0.439906  0.449719  0.421898   

feature                                                 ...       zcr  \
statistics                                              ...       std   
feature_number                                          ...        01   
song_pos          pos_07    pos_08    pos_09    pos_10  ...    pos_02   
track_id                                                ...             
3               0.482706  0.474355  0.573654  0.513427  ...  0.057437   
134             0.421602  0.437058  0.451682  0.456216  ...  0.063500   
136             0.220359  0.307832  0.263045  0.363216  ...  0.008543   
139             0.473008  0.495474  0.437456  0.467358  ...  0.027704   
181             0.414196  0.467424  0.431753  0.399964  ...  0.033205   

feature                                                                     \
statistics                                                                   
feature_number                                                               
song_pos          pos_03    pos_04    pos_05    pos_06    pos_07    pos_08   
track_id                                                                     
3               0.076173  0.076854  0.089157  0.055686  0.035806  0.013286   
134             0.042880  0.046147  0.050985  0.045815  0.037073  0.066570   
136             0.006473  0.008653  0.009003  0.008523  0.008338  0.009464   
139             0.028357  0.026644  0.020512  0.021097  0.016603  0.016951   
181             0.021251  0.032047  0.017333  0.019791  0.023580  0.021319   

feature                                track  
statistics                         genre_top  
feature_number                                
song_pos          pos_09    pos_10            
track_id                                      
3               0.039492  0.067383         7  
134             0.045812  0.062895         7  
136             0.008612  0.008036        13  
139             0.011645  0.018713         6  
181             0.028439  0.033835        13  

[5 rows x 3701 columns]

In [56]:
# Remove corrupted rows (rows with NaNs) - 15 rows
#na_rows = fma_med[fma_med.isna().any(axis=1)]
fma_med.dropna(inplace=True)
print(fma_med.shape)
fma_med.head()

(16985, 3701)


feature        chroma_cens                                                    \
statistics             max                                                     
feature_number          01                                                     
song_pos            pos_01    pos_02    pos_03    pos_04    pos_05    pos_06   
track_id                                                                       
3                 0.417433  0.350428  0.376733  0.508503  0.443255  0.581191   
134               0.446382  0.442526  0.419414  0.433643  0.407400  0.501106   
136               0.275723  0.390478  0.269860  0.068231  0.104821  0.126996   
139               0.286037  0.731147  0.536617  0.612791  0.633770  0.646182   
181               0.401898  0.413877  0.458279  0.439906  0.449719  0.421898   

feature                                                 ...       zcr  \
statistics                                              ...       std   
feature_number                                          ...        01   
song_pos          pos_07    pos_08    pos_09    pos_10  ...    pos_02   
track_id                                                ...             
3               0.482706  0.474355  0.573654  0.513427  ...  0.057437   
134             0.421602  0.437058  0.451682  0.456216  ...  0.063500   
136             0.220359  0.307832  0.263045  0.363216  ...  0.008543   
139             0.473008  0.495474  0.437456  0.467358  ...  0.027704   
181             0.414196  0.467424  0.431753  0.399964  ...  0.033205   

feature                                                                     \
statistics                                                                   
feature_number                                                               
song_pos          pos_03    pos_04    pos_05    pos_06    pos_07    pos_08   
track_id                                                                     
3               0.076173  0.076854  0.089157  0.055686  0.035806  0.013286   
134             0.042880  0.046147  0.050985  0.045815  0.037073  0.066570   
136             0.006473  0.008653  0.009003  0.008523  0.008338  0.009464   
139             0.028357  0.026644  0.020512  0.021097  0.016603  0.016951   
181             0.021251  0.032047  0.017333  0.019791  0.023580  0.021319   

feature                                track  
statistics                         genre_top  
feature_number                                
song_pos          pos_09    pos_10            
track_id                                      
3               0.039492  0.067383         7  
134             0.045812  0.062895         7  
136             0.008612  0.008036        13  
139             0.011645  0.018713         6  
181             0.028439  0.033835        13  

[5 rows x 3701 columns]

In [55]:
# See label encodings for classes
print(pd.Series(label2idx.classes_))

0                   Blues
1               Classical
2                 Country
3          Easy Listening
4              Electronic
5            Experimental
6                    Folk
7                 Hip-Hop
8            Instrumental
9           International
10                   Jazz
11    Old-Time / Historic
12                    Pop
13                   Rock
14               Soul-RnB
15                 Spoken
dtype: object


In [58]:
# Split data into train, valid, test set
train = fma_med[tracks_med["set", "split"] == "training"]
valid = fma_med[tracks_med["set", "split"] == "validation"]
test = fma_med[tracks_med["set", "split"] == "test"]
print(train.shape, valid.shape, test.shape)

<ipython-input-58-5b84eb68f24b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train = fma_med[tracks_med["set", "split"] == "training"]
<ipython-input-58-5b84eb68f24b>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid = fma_med[tracks_med["set", "split"] == "validation"]
<ipython-input-58-5b84eb68f24b>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test = fma_med[tracks_med["set", "split"] == "test"]


(13509, 3701) (1704, 3701) (1772, 3701)


In [67]:
# XGBoost
chosen_features = (slice(None), slice(None), slice(None), slice(None))
X_train = train.loc[:, chosen_features].drop(("track", "genre_top", "", ""), axis=1)
X_val = valid.loc[:, chosen_features].drop(("track", "genre_top", "", ""), axis=1)
X_test = test.loc[:, chosen_features].drop(("track", "genre_top", "", ""), axis=1)
y_train = train.loc[:, ("track", "genre_top", "", "")]
y_val = valid.loc[:, ("track", "genre_top", "", "")]
y_test = test.loc[:, ("track", "genre_top", "", "")]
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(13509, 3700) (13509,) (1704, 3700) (1704,) (1772, 3700) (1772,)


In [75]:
# Model assessment function
def model_assess(model, title="Default"):
    model.fit(X_train,y_train,
              eval_set=[(X_val, y_val)],
              verbose=True)
    y_train_prob = model.predict_proba(X_train)
    y_train_pred = model.predict(X_train)
    y_valid_prob = model.predict_proba(X_val)
    y_valid_pred = model.predict(X_val)

    train_loss = log_loss(y_train, y_train_prob)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred, labels=np.arange(len(label2idx.classes_)), average="macro")
    valid_loss = log_loss(y_val, y_valid_prob)
    valid_acc = accuracy_score(y_val, y_valid_pred)
    valid_f1 = f1_score(y_val, y_valid_pred, labels=np.arange(len(label2idx.classes_)), average="macro")
    print("title")
    print(f"Train loss {train_loss:>8.3f}, accuracy {train_acc:>4.3f}, F1 {train_f1:>4.3f} "
    f"| Valid loss {valid_loss:>8.3f}, accuracy {valid_acc:4.3f}, F1 {valid_f1:>4.3f}")

In [ ]:
xgb = XGBClassifier(
    device="cuda",
    early_stopping_rounds=25,
    n_estimators=1000,
    objective="multi:softmax",
    learning_rate=0.1,
    reg_alpha = 0.1,
    seed=42
)
model_assess(xgb, "Cross Gradient Booster with GPUs_Short")

In [114]:
def model_assess(model, weights, title="Default"):
    model.fit(X_train,y_train,
              eval_set=[(X_val, y_val)],
              verbose=True,
              sample_weight=weights)
    y_train_prob = model.predict_proba(X_train)
    y_train_pred = model.predict(X_train)
    y_valid_prob = model.predict_proba(X_val)
    y_valid_pred = model.predict(X_val)

    train_loss = log_loss(y_train, y_train_prob)
    train_acc = accuracy_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred, labels=np.arange(len(label2idx.classes_)), average="macro")
    valid_loss = log_loss(y_val, y_valid_prob)
    valid_acc = accuracy_score(y_val, y_valid_pred)
    valid_f1 = f1_score(y_val, y_valid_pred, labels=np.arange(len(label2idx.classes_)), average="macro")
    print("title")
    print(f"Train loss {train_loss:>8.3f}, accuracy {train_acc:>4.3f}, F1 {train_f1:>4.3f} "
    f"| Valid loss {valid_loss:>8.3f}, accuracy {valid_acc:4.3f}, F1 {valid_f1:>4.3f}")

In [116]:
# Weighing to address class imbalances - using automatic weight calculator

xgb = XGBClassifier(
    device="cuda",
    early_stopping_rounds=25,
    n_estimators=1000,
    objective="multi:softmax",
    learning_rate=0.1,
    reg_alpha = 0.1,
    seed=42
)

balance_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

model_assess(xgb, balance_weights,"Cross Gradient Booster with GPUs_Short")

[0]	validation_0-mlogloss:2.56046
[1]	validation_0-mlogloss:2.40534
[2]	validation_0-mlogloss:2.28590
[3]	validation_0-mlogloss:2.18453
[4]	validation_0-mlogloss:2.09303
[5]	validation_0-mlogloss:2.01683
[6]	validation_0-mlogloss:1.94540
[7]	validation_0-mlogloss:1.88396
[8]	validation_0-mlogloss:1.82967
[9]	validation_0-mlogloss:1.78060
[10]	validation_0-mlogloss:1.73441
[11]	validation_0-mlogloss:1.69278
[12]	validation_0-mlogloss:1.65121
[13]	validation_0-mlogloss:1.61585
[14]	validation_0-mlogloss:1.58118
[15]	validation_0-mlogloss:1.54812
[16]	validation_0-mlogloss:1.51748
[17]	validation_0-mlogloss:1.48641
[18]	validation_0-mlogloss:1.45872
[19]	validation_0-mlogloss:1.43396
[20]	validation_0-mlogloss:1.40997
[21]	validation_0-mlogloss:1.38706
[22]	validation_0-mlogloss:1.36473
[23]	validation_0-mlogloss:1.34429
[24]	validation_0-mlogloss:1.32368
[25]	validation_0-mlogloss:1.30434
[26]	validation_0-mlogloss:1.28686
[27]	validation_0-mlogloss:1.27120
[28]	validation_0-mlogloss:1.2

In [127]:
# Weighing to address class imbalances - Manual calculation of weights by taking inverse of probability selection

class_counts = np.bincount(y_train)
inverse_frequency = 1 / class_counts
total_classes = len(class_counts)
# Normalize weights
weights = (inverse_frequency / inverse_frequency.sum()) * total_classes

sample_weights = weights[y_train]
model_assess(xgb, sample_weights,"Cross Gradient Booster with GPUs_Short")

[0]	validation_0-mlogloss:2.63348
[1]	validation_0-mlogloss:2.51833
[2]	validation_0-mlogloss:2.41660
[3]	validation_0-mlogloss:2.32311
[4]	validation_0-mlogloss:2.24260
[5]	validation_0-mlogloss:2.17687
[6]	validation_0-mlogloss:2.10864
[7]	validation_0-mlogloss:2.04862
[8]	validation_0-mlogloss:1.99518
[9]	validation_0-mlogloss:1.94515
[10]	validation_0-mlogloss:1.90090
[11]	validation_0-mlogloss:1.85735
[12]	validation_0-mlogloss:1.81785
[13]	validation_0-mlogloss:1.77974
[14]	validation_0-mlogloss:1.74449
[15]	validation_0-mlogloss:1.70946
[16]	validation_0-mlogloss:1.68062
[17]	validation_0-mlogloss:1.65089
[18]	validation_0-mlogloss:1.62316
[19]	validation_0-mlogloss:1.59614
[20]	validation_0-mlogloss:1.57165
[21]	validation_0-mlogloss:1.54785
[22]	validation_0-mlogloss:1.52695
[23]	validation_0-mlogloss:1.50596
[24]	validation_0-mlogloss:1.48422
[25]	validation_0-mlogloss:1.46502
[26]	validation_0-mlogloss:1.44612
[27]	validation_0-mlogloss:1.42889
[28]	validation_0-mlogloss:1.4

In [ ]:
# Cross-Validation

param_grid = {
    "max_depth": [2, 3, 4, 6, 9],
    "learning_rate": [0.001, 0.01, 0.1, 0.2],
    "gamma": [0, 0.25, 0.5, 0.75, 1],
    "subsample": [0.75, 0.9],
    "colsample_bytree": [0.25, 0.5, 0.75],
    "reg_alpha": [0, 0.1, 0.2],
    "reg_lambda": [0, 0.1, 0.2],
}

model = XGBClassifier(
    device="cuda",
    early_stopping_rounds=25,
    n_estimators=1000,
    objective="multi:softmax",
)

grid = GridSearchCV(
    estimator=model,
    scoring="neg_log_loss",
    param_grid=param_grid,
    cv=5,
    verbose=1,
    n_jobs=-1,
)


grid.fit(X_train, y_train, eval_set=[(X_val, y_val)])
print("Best Parameters:", grid.best_params_)